<a href="https://colab.research.google.com/github/BrMrtn/GoogleColab/blob/main/K%C3%A9pfeldolgoz%C3%A1s/khf4_vintage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
!wget https://raw.githubusercontent.com/BrMrtn/GoogleColab/main/resources/me.jpg

--2024-12-03 21:42:54--  https://raw.githubusercontent.com/BrMrtn/GoogleColab/main/resources/me.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186400 (182K) [image/jpeg]
Saving to: ‘me.jpg.1’

me.jpg.1            100%[===================>] 182.03K  --.-KB/s    in 0.1s    

2024-12-03 21:42:55 (1.26 MB/s) - ‘me.jpg.1’ saved [186400/186400]



In [ ]:
# Kép betöltése
img = cv2.imread('me.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Képszín megváltoztatása


# Vignettálás

# Gyenge kontraszt

# Zaj (folt, hajtás, szakadás) hozzáadása

# Képkeret